# Exploratory data analysis of the tech survey results data set

In [1]:
import sys
import time

import matplotlib.pyplot as plt

%matplotlib inline
import numpy as np
import pandas as pd
from IPython.display import HTML

sys.path.append("code/.")

import mglearn
from IPython.display import display
# from plotting_functions import *

# Classifiers and regressors
from sklearn.dummy import DummyClassifier, DummyRegressor

# Preprocessing and pipeline
from sklearn.impute import SimpleImputer

# train test split and cross validation
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    MinMaxScaler,
    OneHotEncoder,
    OrdinalEncoder,
    StandardScaler,
)
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier


In [2]:
tech_df = pd.read_csv('../data/raw/survey_results_ca_usa.csv')

In [3]:
tech_df.shape

(11600, 48)

In [4]:
tech_df.shape[0] - tech_df.dropna().shape[0]

11600

# Summary of the data set
The data set used in this project is from the salary and work information survey created by Stack Overflow in 2019. Each row in the data set represents work related information of a software developer, including the job title, education level, location, years of experience and technology, etc. All the data was collected from the online survey in 2019. There are 11600 observations in the data set, and 48 features. All the observations have at least one missing values in the data set. Below we show the number of missing values for each feature in the data set.

In [5]:
pd.DataFrame(tech_df.isnull().sum()).rename(columns = {0:'missing_value_counts'})

,missing_value_counts
ResponseId,0
MainBranch,0
Employment,4
Country,0
US_State,1808
UK_Country,11600
EdLevel,4
Age1stCode,6
LearnCode,25
YearsCode,55


# Partition the data set into training and test sets
Before proceeding further, we will split the data such that 80% of observations are in the training and 20% of observations are in the test set. Below we list the summary and head as well as tail for the training data set:

In [6]:
pd.set_option("max_columns", 50)
train_df, test_df = train_test_split(tech_df, test_size=0.2, random_state=123)

In [7]:
train_df.describe(include = 'all')

,ResponseId,MainBranch,Employment,Country,US_State,UK_Country,EdLevel,Age1stCode,LearnCode,YearsCode,YearsCodePro,DevType,OrgSize,Currency,CompTotal,CompFreq,LanguageHaveWorkedWith,LanguageWantToWorkWith,DatabaseHaveWorkedWith,DatabaseWantToWorkWith,PlatformHaveWorkedWith,PlatformWantToWorkWith,WebframeHaveWorkedWith,WebframeWantToWorkWith,MiscTechHaveWorkedWith,MiscTechWantToWorkWith,ToolsTechHaveWorkedWith,ToolsTechWantToWorkWith,NEWCollabToolsHaveWorkedWith,NEWCollabToolsWantToWorkWith,OpSys,NEWStuck,NEWSOSites,SOVisitFreq,SOAccount,SOPartFreq,SOComm,NEWOtherComms,Age,Gender,Trans,Sexuality,Ethnicity,Accessibility,MentalHealth,SurveyLength,SurveyEase,ConvertedCompYearly
count,9280.000000,9280,9276,9280,7859,0.0,9278,9276,9263,9237,9221,9263,9267,9280,9.280000e+03,9280,9264,8661,7973,6633,6890,5583,7142,6015,5175,4167,8543,7730,9173,8330,9276,9269,9269,9251,9259,7567,9251,9253,9248,9234,9198,8739,9157,8937,8946,9161,9210,9.280000e+03
unique,NaN,2,5,2,52,NaN,9,9,343,52,52,2096,10,11,NaN,3,5159,4827,1073,903,78,81,1403,1070,392,486,372,515,2211,1509,6,684,7,5,3,6,6,2,8,14,4,24,130,19,30,3,3,NaN
top,NaN,I am a developer by profession,Employed full-time,United States of America,California,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",11 - 17 years,School,10,5,"Developer, full-stack","10,000 or more employees",USD\tUnited States dollar,NaN,Yearly,C#;HTML/CSS;JavaScript;SQL,Rust,Microsoft SQL Server,PostgreSQL,AWS,AWS,React.js,React.js,.NET Framework;.NET Core / .NET 5,.NET Core / .NET 5,Git,Git,Visual Studio Code,Visual Studio Code,MacOS,Call a coworker or friend;Visit Stack Overflow...,Stack Overflow;Stack Exchange,A few times per week,Yes,Less than once per month or monthly,"No, not really",No,25-34 years old,Man,No,Straight / Heterosexual,White or of European descent,None of the above,None of the above,Appropriate in length,Easy,NaN
freq,NaN,8402,8623,7924,1039,NaN,5257,5179,1025,712,708,1440,1893,7952,NaN,8401,96,119,709,670,2033,1548,476,558,1233,750,2444,1696,773,1151,3669,403,6678,2852,7574,3960,3122,5639,4064,8328,8862,7365,6817,8517,5671,7989,7762,NaN
mean,41178.040302,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.512036e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.447965e+05
std,24510.640982,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.165809e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.930823e+05
min,18.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+00
25%,19769.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.200000e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.137500e+04
50%,44456.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.170000e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.200000e+05
75%,60547.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.598125e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.646250e+05


In [8]:
train_df.head()

,ResponseId,MainBranch,Employment,Country,US_State,UK_Country,EdLevel,Age1stCode,LearnCode,YearsCode,YearsCodePro,DevType,OrgSize,Currency,CompTotal,CompFreq,LanguageHaveWorkedWith,LanguageWantToWorkWith,DatabaseHaveWorkedWith,DatabaseWantToWorkWith,PlatformHaveWorkedWith,PlatformWantToWorkWith,WebframeHaveWorkedWith,WebframeWantToWorkWith,MiscTechHaveWorkedWith,MiscTechWantToWorkWith,ToolsTechHaveWorkedWith,ToolsTechWantToWorkWith,NEWCollabToolsHaveWorkedWith,NEWCollabToolsWantToWorkWith,OpSys,NEWStuck,NEWSOSites,SOVisitFreq,SOAccount,SOPartFreq,SOComm,NEWOtherComms,Age,Gender,Trans,Sexuality,Ethnicity,Accessibility,MentalHealth,SurveyLength,SurveyEase,ConvertedCompYearly
7966,57095,I am a developer by profession,Employed full-time,United States of America,California,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",11 - 17 years,School;Online Forum;Books / Physical media,24,15,"Developer, full-stack","1,000 to 4,999 employees",USD\tUnited States dollar,500000,Yearly,Bash/Shell;Java;JavaScript;Kotlin;Node.js;Obje...,JavaScript;Kotlin;Node.js;Python;SQL;Swift;Typ...,MySQL;Redis;SQLite,Firebase;Redis;SQLite,AWS;Google Cloud Platform,Google Cloud Platform,Angular.js;React.js,React.js,NaN,NaN,Ansible;Docker;Git;Kubernetes;Puppet;Terraform,Ansible;Docker;Git;Kubernetes;Terraform,Android Studio;Atom;IntelliJ;IPython/Jupyter;V...,Android Studio;Atom;IntelliJ;IPython/Jupyter;V...,MacOS,Visit Stack Overflow;Go for a walk or other ph...,Stack Overflow;Stack Exchange;Stack Overflow f...,A few times per month or weekly,Yes,Less than once per month or monthly,Neutral,No,35-44 years old,Man,No,Straight / Heterosexual,White or of European descent,None of the above,None of the above,Appropriate in length,Easy,500000
10813,77069,I am a developer by profession,Employed full-time,Canada,NaN,NaN,Some college/university study without earning ...,18 - 24 years,Coding Bootcamp;Online Courses or Certificatio...,6,4,"Developer, full-stack;Developer, back-end",500 to 999 employees,CAD\tCanadian dollar,100000,Yearly,JavaScript;Node.js;Ruby,Clojure;Elixir;Go;Haskell;Rust,MySQL;PostgreSQL;Redis;SQLite,NaN,AWS;DigitalOcean;Google Cloud Platform,NaN,Ruby on Rails,React.js,NaN,NaN,Ansible;Docker;Git;Terraform,Kubernetes,Vim,NaN,Linux-based,Visit Stack Overflow;Go for a walk or other ph...,Stack Overflow,Daily or almost daily,Yes,I have never participated in Q&A on Stack Over...,"Yes, somewhat",Yes,18-24 years old,Man,No,Straight / Heterosexual,White or of European descent,None of the above,None of the above,Appropriate in length,Neither easy nor difficult,75631
9452,66495,I am a developer by profession,Employed full-time,United States of America,Washington,NaN,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",11 - 17 years,School;Books / Physical media,17,9,"Developer, full-stack;Engineer, data;Data scie...",20 to 99 employees,USD\tUnited States dollar,275000,Yearly,Bash/Shell;Clojure;Java;JavaScript;Node.js;Sca...,Clojure;Rust;SQL,DynamoDB;Elasticsearch;PostgreSQL,PostgreSQL,AWS,AWS;Google Cloud Platform,React.js,React.js,NaN,NaN,Docker;Git,Docker;Git;Kubernetes,IntelliJ;Visual Studio Code,IntelliJ;Visual Studio Code,MacOS,Call a coworker or friend;Go for a walk or oth...,Stack Overflow,A few times per week,Yes,Less than once per month or monthly,"No, not really",No,25-34 years old,Man,No,Straight / Heterosexual,White or of European descent,None of the above,I have a mood or emotional disorder (e.g. depr...,Appropriate in length,Easy,275000
10338,73864,I am a developer by profession,Employed full-time,United States of America,Nebraska,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",11 - 17 years,School;Books / Physical media,23,20,"Developer, front-end;Developer, full-stack","1,000 to 4,999 employees",USD\tUnited States dollar,150000,Yearly,HTML/CSS;Java;JavaScript;Node.js;SQL;TypeScript,HTML/CSS;Java;JavaScript;Node.js;SQL;TypeScript,MySQL,MongoDB;MySQL,NaN,NaN,Angular;Express;React.js;Spring,Angular;Express;Spring,NaN,NaN,Docker;Git,Git,IntelliJ;NetB

In [9]:
train_df.tail()

,ResponseId,MainBranch,Employment,Country,US_State,UK_Country,EdLevel,Age1stCode,LearnCode,YearsCode,YearsCodePro,DevType,OrgSize,Currency,CompTotal,CompFreq,LanguageHaveWorkedWith,LanguageWantToWorkWith,DatabaseHaveWorkedWith,DatabaseWantToWorkWith,PlatformHaveWorkedWith,PlatformWantToWorkWith,WebframeHaveWorkedWith,WebframeWantToWorkWith,MiscTechHaveWorkedWith,MiscTechWantToWorkWith,ToolsTechHaveWorkedWith,ToolsTechWantToWorkWith,NEWCollabToolsHaveWorkedWith,NEWCollabToolsWantToWorkWith,OpSys,NEWStuck,NEWSOSites,SOVisitFreq,SOAccount,SOPartFreq,SOComm,NEWOtherComms,Age,Gender,Trans,Sexuality,Ethnicity,Accessibility,MentalHealth,SurveyLength,SurveyEase,ConvertedCompYearly
9785,69393,I am a developer by profession,Employed full-time,United States of America,Texas,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",18 - 24 years,Coding Bootcamp;Friend or family member;Books ...,1,Less than 1 year,"Developer, full-stack",100 to 499 employees,USD\tUnited States dollar,70000,Yearly,C++;Dart;HTML/CSS;JavaScript;Node.js;SQL,Clojure;Java,Firebase;MongoDB;MySQL;Redis,NaN,Google Cloud Platform;Heroku;Microsoft Azure,AWS,ASP.NET Core ;jQuery;React.js,Angular;Vue.js,.NET Core / .NET 5;Flutter;TensorFlow,Torch/PyTorch,Docker;Git;Kubernetes,NaN,Visual Studio;Visual Studio Code,NaN,Windows,Call a coworker or friend;Visit Stack Overflow,Stack Overflow,Multiple times per day,No,NaN,"No, not really",No,18-24 years old,Woman,No,Straight / Heterosexual,White or of European descent;South Asian;Biracial,None of the above,None of the above,Appropriate in length,Easy,70000
7763,56274,I am a developer by profession,Employed full-time,United States of America,California,NaN,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",11 - 17 years,School,26,21,"Developer, full-stack;Product manager;Senior E...",20 to 99 employees,USD\tUnited States dollar,325000,Yearly,HTML/CSS;JavaScript;Ruby;SQL,Elixir;Node.js;Rust;TypeScript,Elasticsearch;PostgreSQL;Redis,Firebase,AWS;DigitalOcean;Google Cloud Platform;Heroku,NaN,Gatsby;jQuery;React.js;Ruby on Rails,NaN,NaN,NaN,Git;Yarn,Kubernetes;Terraform,Emacs;Visual Studio Code,NaN,MacOS,Visit Stack Overflow;Google it,Stack Overflow;Stack Exchange,A few times per week,Yes,I have never participated in Q&A on Stack Over...,"Yes, somewhat",No,35-44 years old,Man,No,Straight / Heterosexual,East Asian,None of the above,None of the above,Appropriate in length,Easy,325000
5218,36475,I am a developer by profession,Employed full-time,United States of America,Washington,NaN,"Other doctoral degree (Ph.D., Ed.D., etc.)",18 - 24 years,"Other online resources (ex: videos, blogs, etc...",44,46,"Developer, front-end;Developer, desktop or ent...","1,000 to 4,999 employees",USD\tUnited States dollar,125000,Monthly,C#;Haskell;TypeScript,C#;Erlang;Haskell;Node.js;TypeScript,Microsoft SQL Server,Microsoft SQL Server;MongoDB;Redis,AWS,NaN,Angular;ASP.NET;ASP.NET Core ;Flask;Vue.js,Angular;ASP.NET Core ;Vue.js,.NET Framework;.NET Core / .NET 5,.NET Core / .NET 5,Git,Deno;Git,Notepad++;PyCharm;Visual Studio;Visual Studio ...,Notepad++;PyCharm;Visual Studio;Visual Studio ...,Windows,Visit Stack Overflow;Google it;Visit another d...,Stack Overflow;Stack Exchange,Multiple times per day,Yes,A few times per month or weekly,Neutral,No,65 years or older,Man,No,Straight / Heterosexual,White or of European descent,I am deaf / hard of hearing,None of the above,Appropriate in length,Easy,1500000
1346,7260,"I am not primarily a developer, but I write co...",Employed full-time,United States of America,Colorado,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",11 - 17 years,"Other online resources (ex: videos, blogs, etc...",12,12,"Engineer, data;Data scientist or machine learn...","1,000 to 4,999 employees",USD\tUnited States dollar,100000,Yearly,Bash/Shell;Python;R;SQL,Bash/Shell;Python;R;SQL,MariaDB;MySQL;PostgreSQL;SQLite,PostgreSQL,AWS,AWS,NaN,NaN,Keras;NumPy;Pandas;TensorFlow,Keras;NumPy;Pandas;TensorFlow,Git,Git,IPython/Jupyter;PyCharm;RStudio;Vim,IPyt

In [10]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9280 entries, 7966 to 3582
Data columns (total 48 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ResponseId                    9280 non-null   int64  
 1   MainBranch                    9280 non-null   object 
 2   Employment                    9276 non-null   object 
 3   Country                       9280 non-null   object 
 4   US_State                      7859 non-null   object 
 5   UK_Country                    0 non-null      float64
 6   EdLevel                       9278 non-null   object 
 7   Age1stCode                    9276 non-null   object 
 8   LearnCode                     9263 non-null   object 
 9   YearsCode                     9237 non-null   object 
 10  YearsCodePro                  9221 non-null   object 
 11  DevType                       9263 non-null   object 
 12  OrgSize                       9267 non-null   object 
 13  

# Counts of observations for each feature
Since we only want to look into the top five significant features of salary prediction, it is good to know the counts distribution for each feature. The tables below show the counts of different role which is quite imbalanced.

In [11]:
pd.DataFrame(train_df["MainBranch"].value_counts()).rename(columns = {'MainBranch':'counts'})

,counts
I am a developer by profession,8402
"I am not primarily a developer, but I write code sometimes as part of my work",878


There are 7924 observations from United States of America and 1356 observations from Canada which is also imbalanced here.

In [12]:
pd.DataFrame(train_df["Country"].value_counts()).rename(columns = {'Country':'counts'})

,counts
United States of America,7924
Canada,1356


In education level feature, we can see most developers have hold a bachelor's degree.

In [13]:
pd.DataFrame(train_df["EdLevel"].value_counts()).rename(columns = {'EdLevel':'counts'})

,counts
"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",5257
"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",1669
Some college/university study without earning a degree,1187
"Associate degree (A.A., A.S., etc.)",439
"Other doctoral degree (Ph.D., Ed.D., etc.)",372
"Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)",197
Something else,74
"Professional degree (JD, MD, etc.)",50
Primary/elementary school,33


Experience is important when considering the salary of a developer, the table below shows the counts of developer having different years of coding experience.

In [14]:
yearcode_df = pd.DataFrame(train_df["YearsCode"].value_counts()).reset_index()#.rename(columns={"index":"code_years", "YearsCode":"count"}, inplace=True)
yearcode_df = yearcode_df.rename(columns={"index":"code_years", "YearsCode":"count"})
yearcode_df = (
                yearcode_df[(yearcode_df['code_years'] != "Less than 1 year") & (yearcode_df['code_years'] != "More than 50 years")]
               .apply(pd.to_numeric).sort_values(by = 'code_years').set_index('code_years')
              )#.set_index('code_years').apply(pd.to_numeric).sort_index(inplace=True)
yearcode_df

,count
code_years,
1,27
2,88
3,147
4,242
5,423
6,478
7,469
8,489
9,338


Coding language might also have effect on the target salary, the table shows the counts of developer using different programming languages. We can see Javascript is the most frequently used language for developers.

In [15]:
language_df = pd.DataFrame(train_df["LanguageHaveWorkedWith"].str.split(';',expand=True).apply(pd.value_counts).sum(axis=1))
language_df 

,0
APL,34.0
Assembly,376.0
Bash/Shell,3610.0
C,1360.0
C#,2992.0
C++,1705.0
COBOL,47.0
Clojure,284.0
Crystal,67.0
Dart,224.0
